In [ ]:
dynamic monte carlo

finance: focus on optimal stopping, am or ac
ls+tsi: mc for optimal stopping
several: pde for am
seq dec: stoch control>pde low dim
    
rl: focus on seq dec
dyn prg = pde too slow
sample appr
value method = mc = ls
pg also exists in finance, see eg jaeckel

finance requires high reliability+prec+speed (for intraday risk calcs) in a constantly changing model (because mkt data change)

new appr: dyn mc
compare to heuristic (geodesic) and dqn (stable baseline)


In [71]:
import numpy as np
from numpy.linalg import eig
import pandas as pd
import py_lets_be_quickly_rational
import time
from scipy import interpolate
from scipy.stats import norm
from scipy.optimize import brentq

0) intro
1) review > only optimal stopping
2) model > stoch correl; no resim because chance player always the same; but exploration question remains; check with deepmind paper
2) geodesic+grad descent
3) dqn
4) mcts

In [72]:
def BS(F,K,T,vol):
    d1=(np.log(F/K)+0.5*vol**2*T)/(vol*T**0.5)
    d2=d1-vol*T**0.5
    return F*norm.cdf(d1) - K*norm.cdf(d2)

def IV(price,F,K,T):
    return brentq(lambda v: BS(F,K,T,v)-price, 0.01, 0.5)

In [62]:
# s0, v0, kappa, theta, alpha=vov, rho
modelParam = np.array([
    [1.0, 0.2**2, 4.0, 0.25**2, 0.5, -0.5],
    [1.0, 0.35**2, 3.0, 0.25**2, 0.4, -0.6],
    [1.0, 0.3**2, 2.0, 0.3**2, 0.3, -0.7],
])

n = len(modelParam)

# constraint 2 theta vbar > alpha**2
for i in range(n):
    _, _, theta, vbar, alpha, _ = modelParam[i]
    print(2*theta*vbar-alpha**2)

0.25
0.21499999999999997
0.27


In [63]:
C = np.array([
    [1.0, 0.7, 0.3],
    [0.7, 1.0, 0.5],
    [0.3, 0.5, 1.0]
])

L = np.linalg.cholesky(C)
print(L.T)
print(np.matmul(L, L.T))
print(C-np.matmul(L, L.T))
    
for i in range(3):
    b=np.random.multivariate_normal(np.zeros(3),0.02**2*np.eye(3))
    a = L[i,:] + b
    L[i,:] = a / np.linalg.norm(a)
print(np.matmul(L, L.T))

[[1.         0.7        0.3       ]
 [0.         0.71414284 0.40608122]
 [0.         0.         0.86319062]]
[[1.  0.7 0.3]
 [0.7 1.  0.5]
 [0.3 0.5 1. ]]
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  1.11022302e-16  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -2.22044605e-16]]
[[1.         0.64179985 0.29237176]
 [0.64179985 1.         0.49247261]
 [0.29237176 0.49247261 1.        ]]


In [64]:
def xi(a,b,c,d,k):
    r1 = np.cos(k*np.pi*(d-a)/(b-a))*np.exp(d) - np.cos(k*np.pi*(c-a)/(b-a))*np.exp(c)
    r2 = k*np.pi/(b-a)*np.sin(k*np.pi*(d-a)/(b-a))*np.exp(d) - k*np.pi/(b-a)*np.sin(k*np.pi*(c-a)/(b-a))*np.exp(c)
    return (r1+r2) / (1+(k*np.pi/(b-a))**2)

def vpsi(a,b,c,d,ks):
    ks = np.maximum(ks,1)
    r = (np.sin(ks*np.pi*(d-a)/(b-a)) - np.sin(ks*np.pi*(c-a)/(b-a)))*(b-a)/(ks*np.pi)
    r[0] = d-c
    return r

def vv_put_orig(a, b, ks):
    r = 2 / (b-a) * (-xi(a,b,a,0,ks)+vpsi(a,b,a,0,ks))
    r[0] = r[0] / 2.0
    return r

def phi_heston2_pre(omega, kappa,theta, alpha, rho,T):
    z=omega
    tau=T
    v=V0
    vbar=theta
    lamb=kappa
    eta=alpha
    a0=lamb-rho*eta*z*1j
    gamma=np.power(eta**2*(z**2+z*1j)+a0*a0, 0.5)
    G=(a0-gamma)/(a0+gamma)
    a1=1.0/eta/eta*((1-np.exp(-gamma*tau))/(1-G*np.exp(-gamma*tau)))*(a0-gamma)
    a2=lamb*vbar/eta/eta*(tau*(a0-gamma)-2*np.log((1-G*np.exp(-gamma*tau))/(1-G))) #c*vbar
    return a1, a2

def phi_heston2(omega, kappa,theta, alpha, rho,V0,T):
    a1, a2 = phi_heston2_pre(omega, kappa,theta, alpha, rho,T)
    return np.exp(V0*a1+a2)

def phi_heston2_post(a,V0):
    a1, a2 = a
    return np.exp(V0*a1+a2)

def cumulants(kappa, theta, V0, t):
    c1 = (1-np.exp(-kappa*t))*(theta-V0)/(2*kappa)-theta*t/2
    c21 = V0 / (4*kappa**3) * (4*kappa**2*(1+(rho*alpha*t-1)*np.exp(-kappa*t))
                             +kappa*(4*rho*alpha*(np.exp(-kappa*t)-1)-2*alpha**2*t*np.exp(-kappa*t))
                             +alpha**2*(1-np.exp(-2*kappa*t)))
    c22 = theta / (8*kappa**3) * (8*kappa**3*t - 8*kappa**2*(1+rho*alpha*t+(rho*alpha*t-1)*np.exp(-kappa*t))+\
                                  2*kappa*((1+2*np.exp(-kappa*t))*alpha**2*t+ 8*(1-np.exp(-kappa*t))*rho*alpha)+\
                                  alpha**2*(np.exp(-2*kappa*t)+4*np.exp(-kappa*t)-5))
    return c1, c21+c22

In [65]:
T=1
S0, V0, kappa, theta, alpha, rho = modelParam[1,:]
c1, c2 = cumulants(kappa, theta, V0, T)
c1, c2 = cumulants(kappa, theta, theta, T)
L=12
a=c1 - L*c2**0.5
b=c1 + L*c2**0.5
F=1
N=50

In [69]:
Ks=np.linspace(0.95,1.05,11)
t1=time.perf_counter()
xs=np.log(F/Ks)
ks = np.array(range(N))
xxs = ks*np.pi/(b-a)

h = phi_heston2(xxs, kappa,theta,alpha,rho,V0,T) * vv_put_orig(a,b,ks)
res = np.zeros(len(Ks))
iv = np.zeros(len(Ks))
q=-1 #put
numIt=1
for i in range(len(xs)):
    res[i] = Ks[i] * np.sum(h * np.exp(1j * (xs[i] - a) * xxs)).real
    iv[i] = py_lets_be_quickly_rational.implied_volatility_from_a_transformed_rational_guess_with_limited_iterations(res[i], S0, Ks[i], T, q, numIt)
t2=time.perf_counter()
print("time:", t2-t1)

for i in range(len(Ks)):
    print(Ks[i], res[i], iv[i])
print("calls")
for i in range(len(Ks)):
    print(Ks[i], res[i] + 1 - Ks[i])

time: 0.0023723579943180084
0.95 0.08591642237953069 0.28146785911645855
0.96 0.09042552557983027 0.28042424770038593
0.97 0.09507940096186268 0.27939376688469664
0.98 0.09987816187140033 0.27837633276572715
0.99 0.10482173355633755 0.2773718645600243
1.0 0.10990985425783668 0.27638028438281337
1.01 0.11514207700040564 0.2754015170300439
1.02 0.12051777206784177 0.27443548976228105
1.03 0.12603613014798598 0.2734821320892611
1.04 0.13169616612540092 0.2725413755544852
1.05 0.13749672349738656 0.2716131535196757
calls
0.95 0.13591642237953083
0.96 0.13042552557983034
0.97 0.12507940096186276
0.98 0.11987816187140043
0.99 0.11482173355633751
1.0 0.10990985425783673
1.01 0.10514207700040568
1.02 0.10051777206784185
1.03 0.09603613014798595
1.04 0.09169616612540077
1.05 0.08749672349738646


In [74]:
BS(1,1.05,1,0.2716131535196757), IV(0.08749672349738646,1,1.05,1)

(0.08749672349738646, 0.27161315351967547)

In [77]:
T=2
n_reset=8
n_resetPerYear=n_reset//T
n_simStepsPerReset=365//(n_resetPerYear)
n_simStepsPerYear=n_simStepsPerReset*n_resetPerYear
nt=n_simStepsPerReset*n_reset
T_reset=[n_simStepsPerReset*q for q in range(1,n_reset)]

dt=1.0/n_simStepsPerYear
sqrtDt=dt**0.5
na=C.shape[0]
ns=10
volCorrel=0.02

In [78]:
#simulate correl
U0 = np.linalg.cholesky(C).T
U0 = np.repeat(U0[np.newaxis, :, :], ns, axis=0)
U = np.repeat(U0[np.newaxis, :, :, :], n_reset, axis=0)
for i in range(n_reset):
    A=U0 if i==0 else U[i-1,:,:,:]
    A=A + np.random.multivariate_normal(np.zeros(na),volCorrel**2*np.eye(na),size=(ns,na))
    U[i,:,:,:]=A/np.linalg.norm(A,axis=1).reshape((ns,1,na))
np.matmul(U[-1,0,:,:].T, U[-1,0,:,:])

array([[1.        , 0.67054047, 0.23873326],
       [0.67054047, 1.        , 0.47520003],
       [0.23873326, 0.47520003, 1.        ]])

In [79]:
# show path/evolution of one correlation matrix by three paths over time

In [80]:
W=np.random.normal(size=(n_reset,ns,na,n_simStepsPerReset)) # you could reduce here because S can do a big step
print(U.shape, W.shape)
W1=np.matmul(U,W)
print(W1.shape)

(8, 10, 3, 3) (8, 10, 3, 91)
(8, 10, 3, 91)


In [81]:
rho=np.ones_like(W1)
for i in range(na):
    rho[:,:,i,:]=modelParam[i,-1]
W2 = rho*W1 + np.sqrt(1-np.square(rho)) * np.random.normal(size=(n_reset,ns,na,n_simStepsPerReset))

In [82]:
X=np.zeros((n_reset,ns,na))
V=np.zeros((n_reset,ns,na))

Xr=np.tile(np.log(modelParam[:,0]),(ns,1))
Vr=np.tile(modelParam[:,1],(ns,1))

for i in range(n_reset):
    for j in range(n_simStepsPerReset):
        sqrtV = np.sqrt(Vr)
        Xr = Xr + sqrtV * sqrtDt * W1[i,:,:,j] - 0.5 * Vr * dt
        Vr = np.abs(modelParam[:,2]*(modelParam[:,3]-Vr)*dt + modelParam[:,4]*sqrtV*sqrtDt*W2[i,:,:,j])         
    X[i,:,:] = Xr
    V[i,:,:] = Vr
S=np.exp(X)

In [83]:
np.mean(S[-1,:,:],axis=0)

array([1.00450329, 0.98637827, 0.98530973])

In [100]:
S.shape

(8, 10, 3)

In [ ]:
create w grid
space is [0,0.05,...,0.35]^5 but not all points are used

In [108]:
co=0
n=8
for i in range(n):
    for j in range(n):
        for k in range(n):
            for l in range(n):
                for m in range(n):
                    if (i+j+k+l+m)==20:
                        co+=1
co

2226

In [98]:
vol_ref = np.sqrt(modelParam[:,3])
ms = np.array([-0.5, 0, 0.5])
nk=len(ms)
periodDt = T/n_reset
sqrtPeriodDt=periodDt**0.5

Ks_base = np.exp(np.matmul(vol_ref.reshape((na,1)),ms.reshape((1,-1)))*sqrtPeriodDt)
ks = np.array(range(N))
a=np.zeros(na)
b=np.zeros(na)
vv_po = np.zeros((na,N))
for k in range(na):
    kappa, theta, alpha, rho = modelParam[k,2:]
    c1, c2 = cumulants(kappa, theta, theta, periodDt)
    L=12
    a[k]=c1 - L*c2**0.5
    b[k]=c1 + L*c2**0.5
    vv_po[k] = vv_put_orig(a[k],b[k],ks)
#xxs = ks*np.pi/(b-a)
xxs = np.pi * np.matmul(1/(b-a).reshape(na,1), ks.reshape(1,N))
heston_a = []
for k in range(na):
    kappa, theta, alpha, rho = modelParam[k,2:]
    heston_a.append(phi_heston2_pre(xxs[k], kappa,theta,alpha,rho,periodDt))


t1=time.perf_counter()
Ks = np.zeros((n_reset-1,ns,na,nk))
prices = np.zeros((n_reset-1,ns,na,nk))
vols = np.zeros((n_reset-1,ns,na,nk))
for i in range(n_reset-1):
    for j in range(ns):
        Ks[i,j,:,:]=S[i,j,:].reshape((na,1))*Ks_base
        for k in range(na):
            Sk = S[i,j,k]
            Vk = V[i,j,k]
            kappa, theta, alpha, rho = modelParam[k,2:]
            #use vbar as a time independent approx
#             c1, c2 = cumulants(kappa, theta, theta, periodDt)
#             L=12
#             a=c1 - L*c2**0.5
#             b=c1 + L*c2**0.5
            
            xs=np.log(1.0/Ks_base[k,:])
#             ks = np.array(range(N))
#             xxs = ks*np.pi/(b[k]-a[k])

            #h = phi_heston2(xxs[k], kappa,theta,alpha,rho,Vk, periodDt) * vv_po[k] #vv_put_orig(a[k],b[k],ks)
            h = phi_heston2_post(heston_a[k],Vk) * vv_po[k] #vv_put_orig(a[k],b[k],ks)
            res = np.zeros(len(Ks))
            iv = np.zeros(len(Ks))
            q=-1 #put
            numIt=1
            for l in range(len(xs)):
                prices[i,j,k,l] = Ks[i,j,k,l] * np.sum(h * np.exp(1j * (xs[l] - a[k]) * xxs[k])).real
                vols[i,j,k,l] = py_lets_be_quickly_rational.implied_volatility_from_a_transformed_rational_guess_with_limited_iterations(
                    prices[i,j,k,l], Sk, Ks[i,j,k,l], periodDt, q, numIt)
t2=time.perf_counter()
print("time:", t2-t1)


In [ ]:
parametrisation policy
w, s0, v0, vols, correls


In [ ]:
actions:
    cts too big a step
    increase one, decrease one
    5 linear function

In [ ]:
sigma_atm
A=U[n_rest_i, path_i,:,:] >  corr
A = A * eye(sigma_atm t**0.5)
Si(0)
Sbar=sum wi Si
kappa=ln(K/Sbar)
omegai=wi Si / Sbar
Omega = eye(omegai)
zstar=
Kstar=exp(A*zstar)
sigmai
levy formula

In [ ]:
# observations = 
# w1,..,wn,
# S1,..,Sn,
# V1,..Vn,
# sigma1(ms[0]),..sigma1(ms[-1]),...sigman(ms[-1]),
# rho12,..rho1n,rho23,..rho(n-1)n
# mit n = na

K=1
nsi = 0
nri = 0

w = obs[:na]
S = obs[na:2*na]
Sbar = np.dot(w, S)
kappa = math.log(K / Sbar)
omega = w*S/Sbar
Omega = np.diag(omega)

sigma_atm = np.zeros(na)
m_mid=ms//2
for i in range(na):
    sigma_atm[i] = obs[3*na+m_mid+i*len(ms)]
# nein, nicht A berechnen von correlations. 
AVar = np.matmul(A, sqrtPeriodDt*np.diag(sigma_atm))    

C=np.ones((na, na))
c=0
for i in range(1, na):
    C[i-1,i:]=obs(na*(3+len(ms))+c:na*(3+len(ms))+c+na-i)
    C[i:,i-1]=C[i-1,i:].T
    c+=na-i
omegaComega = np.matmul(omega.T, np.matmul(C, omega))
zstar=kappa*np.matmul(A.T, omega) / omegaComega * ( 1 + kappa/2 *
    (np.eye(na) + 2 * np.matmul(A.T, np.matmul(Omega, A)) / omegaComega -
    3*np.matmul(omega.T,npmatmul(C, np.matmul(Omega, np.matmul(C, omega)))) / omegaComega**2 * np.eye(na) ))
Kstar=S*np.exp(np.matmul(A, zstar))
sigma=np.zeros(na)
for i in range(na):
    sigma[i] = interpolate.interp1d(Ks[nri,nsi,i,:], obs[3*na+i*len(ms):3*na+(i+1)*len(ms)])(Kstar[i])
